## Get started

In [10]:
import tensorflow as tf

In [11]:
print('tf version:', tf.__version__)

tf version: 2.0.0-beta1


In [12]:
mnist = tf.keras.datasets.mnist

In [2]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

W0628 10:45:33.518145 4710598080 deprecation.py:323] From /Users/arvid/anaconda3/envs/tf2-cpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 77us/sample - loss: 0.2202 - accuracy: 0.9351
Epoch 2/5
60000/60000 [==============================] - 4s 73us/sample - loss: 0.0968 - accuracy: 0.9703
Epoch 3/5
60000/60000 [==============================] - 4s 75us/sample - loss: 0.0693 - accuracy: 0.9788
Epoch 4/5
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0539 - accuracy: 0.9826
Epoch 5/5
10000/10000 [==============================] - 0s 37us/sample - loss: 0.0688 - accuracy: 0.9798


[0.06881827984197299, 0.9798]